#### 단어 임베딩 : 각 단어를 벡터로 표현. 
####             : - 동음이의어 구분X -> 동일한 벡터값으로 표현.
####             : + 문장 임베딩에 비해 학습 방법 간단.

In [ ]:
# 1. 원-핫 인코딩(one-hot encoding)
# -> 원-핫 벡터 = 희소(sparse) 벡터 = 희소 행렬 : 전체 요소 중 단 하나의 값만 1

### 과정
# pip install konlpy
from konlpy.tag import Komoran
import numpy as np

komoran = Komoran()
text = '오늘 날씨는 구름이 많아요.'

# 명사만 추출
nouns = komoran.nouns(text)
print(nouns)

['오늘', '날씨', '구름']


In [ ]:
# 단어 사전 구축 
dics = {}
for word in nouns:
  if word not in dics.keys():
    dics[word] = len(dics) # 단어별 인덱스 부여
print(dics)

{'오늘': 0, '날씨': 1, '구름': 2}


In [ ]:
# 원-핫 인코딩
nb_classes = len(dics) # 원-핫 벡터 차원의 크기 = 단어 수
targets = list(dics.values())
one_hot_targets = np.eye(nb_classes)[targets]
print(one_hot_targets)

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [11]:
# 2. Word2Vec : 분산 표현(distributed representation) = 밀집(dense) 표현

from gensim.models import Word2Vec
from konlpy.tag import Komoran
import time

def read_review_data(filename):
  with open(filename, 'r') as f:
    data = [line.split('\t') for line in f.read().splitlines()]
    data = data[1:] # 헤더 제외
  return data

# 학습 시간 측정 시작
start = time.time() 

review_data = read_review_data('./ratings.txt')
print(len(review_data)) # 리뷰 데이터 전체 개수
print('소요 시간 : ', time.time() - start) # 초

200000
소요 시간 :  0.6325592994689941


In [ ]:
komoran = Komoran()

# 명사만 추출
docs = [komoran.nouns(sentence[1]) for sentence in review_data]

# 모델 학습
model = Word2Vec(sentences=docs, # 학습에 필요한 문장 데이터, 모델의 입력값
                 size=200,       # 벡터의 차원, 크기
                 window=4,       # 주변 단어 크기 (-+4)
                 hs=1,           # softmax 사용
                 min_count=2,    # 빈도수 2이하의 단어들은 학습X
                 sg=1)           # skip_gram 모델 <-> 0 : CBOW 모델

# 모델 저장
model.save('nvmc_model')

print('학습된 전체 말뭉치 수 : ', model.corpus_count)
print('전체 단어 수 : ', model.corpus_total_words)


학습된 전체 말뭉치 수 :  200000
전체 단어 수 :  1076896


In [12]:
# model = Word2Vec.load('nvmc.model')
print(model.corpus_total_words)

1076896


In [13]:
# '사랑'의 단어 임베딩 벡터
print('사랑 : ', model.wv['사랑'])

사랑 :  [-2.97578931e-01  7.12058619e-02  1.59343913e-01 -1.51612595e-01
  2.77880698e-01  5.68443686e-02  2.88528442e-01 -1.20813280e-01
  2.98586693e-02  1.91238225e-01  8.24682415e-02 -3.96510243e-01
 -2.84748580e-02 -2.38624632e-01 -6.11718670e-02  4.86974865e-01
  1.45292893e-01 -1.07594490e-01  3.11270297e-01  3.00044775e-01
 -2.88162798e-01 -2.40445659e-01  2.01662272e-01 -5.18656187e-02
  1.08743057e-01  1.61261290e-01  3.37705389e-02 -3.46917212e-02
  9.85553786e-02  2.94933736e-01  9.46900749e-04 -1.28874391e-01
  1.01001993e-01 -2.89510228e-02  2.07849219e-01  3.48576345e-02
  2.33263671e-01 -5.19848287e-01 -4.11301851e-02  8.38909894e-02
  3.64438623e-01 -2.88541585e-01  1.14095636e-01  1.17198154e-02
  2.91589409e-01 -2.97565218e-02 -3.33097190e-01 -3.95681202e-01
 -8.73521119e-02 -1.99273422e-01 -9.92501006e-02  1.18523225e-01
  2.13210836e-01 -1.75432637e-02  1.06171139e-01 -1.52044259e-02
 -9.00473893e-02  1.08999759e-01  2.36939535e-01  1.96870845e-02
  6.84565753e-02 -2

In [19]:
# 단어 유사도 계산
print('일요일 = 월요일 :', model.wv.similarity(w1='일요일', w2='월요일'))
print('안성기 = 배우 :', model.wv.similarity(w1='안성기', w2='배우'))
print('대기업 = 삼성 :', model.wv.similarity(w1='대기업', w2='삼성'))
print('일요일 = 삼성 :', model.wv.similarity(w1='일요일', w2='삼성'))
print('히어로 = 삼성 :', model.wv.similarity(w1='히어로', w2='삼성'))

일요일 = 월요일 : 0.6877318
안성기 = 배우 : 0.5518446
대기업 = 삼성 : 0.5976017
일요일 = 삼성 : 0.25114098
히어로 = 삼성 : 0.16577387


In [20]:
# 가장 유사한 단어 추출
print(model.wv.most_similar('안성기', topn=5))
print(model.wv.most_similar('시리즈', topn=5))

[('정려원', 0.7247748374938965), ('장미희', 0.706409215927124), ('박중훈', 0.705795168876648), ('씨야', 0.7056567668914795), ('강수연', 0.6990177035331726)]
[('더 울버린', 0.6559002995491028), ('캐리비안의 해적', 0.6524219512939453), ('비포 선셋', 0.6483879089355469), ('데스티네이션', 0.6392425298690796), ('X맨', 0.6303446292877197)]
